In [1]:
# Bayesian with MCMC - Gibbs sampling
# Simple example: Inference parameter mu (the mean) of a multivariate Normal distribution
import numpy as np
import edward as ed
from edward.models import Empirical, Normal
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Model
n = 5 # Variable dimension
nobs = 30000 # number of observations
ns = 1000 # number of samples
mu = Normal(loc=tf.zeros(n), scale=tf.ones(n)) # This is the parameter to be estimated
x = Normal(loc=mu, scale=tf.ones(n), sample_shape=nobs) # This is the observation variable
qmu = Empirical(tf.Variable(tf.zeros([ns,n]))) # This is the posterior of parameter, which is an empirical distribution with ns samples

In [3]:
# Data
true_mu = np.ones(n) # The true value of the mean
x_data = true_mu + np.random.randn(nobs,n) # Observations: multivariate normal distribution

In [4]:
# Inference by MCMC-Gibbs sampling
inference = ed.Gibbs({mu: qmu}, data={x: x_data})
inference.initialize(n_iter=1000)
tf.global_variables_initializer().run()

for _ in range(inference.n_iter):
    info_dict = inference.update()
    inference.print_progress(info_dict)

inference.finalize()

1000/1000 [100%] ██████████████████████████████ Elapsed: 1s | Acceptance Rate: 1.001

In [5]:
# View estimations
sess = ed.get_session()
samples_mu = [sess.run(qmu) for i in range(1000)]
muest = np.mean(samples_mu, axis=0)
rmse = np.sqrt(np.mean([(np.linalg.norm(samples_mu[i]-true_mu))**2 for i in range(len(samples_mu))]))
print("Estimated mean: ", muest)
print("RMSE: ", rmse)

('Estimated mean: ', array([ 1.00643039,  0.99724209,  0.9980157 ,  1.00478768,  0.99855465], dtype=float32))
('RMSE: ', 0.015888155187349755)
